In [1]:
# Import packages
import pandas as pd
import requests
import os
# import plotly.express as px
# import plotly.graph_objects as go
from datetime import datetime, date
from google.cloud import storage
from bbox import * 
from datetime import date, timedelta



# Fix issue where request will get stuck unless timeout is
# specified (see https://stackoverflow.com/a/76506083)
requests.packages.urllib3.util.connection.HAS_IPV6 = False

# Prevent column truncation in Pandas output
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 500)
pd.set_option("display.width", None)

In [2]:
api_email = 'amanda.murray19@gmail.com'
api_key = 'bluemouse27'
param = "42602" # NO2
range_start = "20230801" # Start of sentinel data
range_end = "20250701" # Current date at time of pull
min_lon = lon_min
max_lon = lon_max
min_lat = lat_min
max_lat = lat_max  

date_format = "%Y%m%d"
date_format_dashes = "%Y-%m-%d"

raw_csv_name = f"aqs_{range_start}_{range_end}_raw.csv.gz"
raw_csv_gcs_path = f"{raw_csv_name}"

start_date = datetime.strptime(range_start, date_format).date()
end_date = datetime.strptime(range_end, date_format).date()
ranges_by_month = [
    (
        max(start_date, date(year, month, 1)).strftime(date_format),
        min(end_date, (date(year, month, 1) + timedelta(days=31)).replace(day=1) - timedelta(days=1)).strftime(date_format)
    )
    for year in range(start_date.year, end_date.year + 1)
    for month in range(1, 13)
    if date(year, month, 1) <= end_date and date(year, month, 1) + timedelta(days=31) >= start_date
]


In [3]:
def aqs_request(bdate, edate):
    aqs_url = "https://aqs.epa.gov/data/api/sampleData/byBox"
    aqs_params = {
        "email": api_email,
        "key": api_key,
        "param": param,
        "bdate": bdate,
        "edate": edate,
        "minlat": min_lat,
        "maxlat": max_lat,
        "minlon": min_lon,
        "maxlon": max_lon,
        "duration":'1'
        
    }

    print(f"Pulling air quality sensor data for range between {bdate} and {edate}...", end=" ")
    r = requests.get(aqs_url, params=aqs_params)
    aqs_json = r.json()
    # print(aqs_json)
    aqs_df = pd.json_normalize(aqs_json["Data"])
    print("Success")
    return aqs_df


In [4]:
aqs_df = pd.concat([aqs_request(bdate, edate) for bdate, edate in ranges_by_month], ignore_index=True)

Pulling air quality sensor data for range between 20230801 and 20230731... Success
Pulling air quality sensor data for range between 20230801 and 20230831... Success
Pulling air quality sensor data for range between 20230901 and 20230930... Success
Pulling air quality sensor data for range between 20231001 and 20231031... Success
Pulling air quality sensor data for range between 20231101 and 20231130... Success
Pulling air quality sensor data for range between 20231201 and 20231231... Success
Pulling air quality sensor data for range between 20240101 and 20240131... Success
Pulling air quality sensor data for range between 20240201 and 20240229... Success
Pulling air quality sensor data for range between 20240301 and 20240331... Success
Pulling air quality sensor data for range between 20240401 and 20240430... Success
Pulling air quality sensor data for range between 20240501 and 20240531... Success
Pulling air quality sensor data for range between 20240601 and 20240630... Success
Pull

In [5]:
len(aqs_df['date_local'].unique())*len(aqs_df['site_number'].unique())*24

836160

In [6]:
aqs_df.to_csv('../../data/sensor_data/hourly_raw_am.csv')

In [7]:
check = pd.read_csv('../../data/sensor_data/hourly_raw_am.csv')
check['date_local'].max()

'2025-05-31'